<a href="https://colab.research.google.com/github/ganbing0630/su3cl3/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyngrok flask flask_ngrok
!pip install line-bot-sdk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 13.9 MB/s eta 0:00:00


In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('ngrok_auth_token'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3.messaging import TextMessage
from linebot.v3.messaging import StickerMessage
from linebot.v3.messaging import ImageMessage
from linebot.v3.messaging import VideoMessage
from linebot.v3.messaging import LocationMessage
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage
 )

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


import google.generativeai as genai

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel("gemini-2.0-flash")


def ask_gemini(question):
  response = model.generate_content(question)
  return response.text


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)




        action = event.message.text
        if action == 'sticker':
          #回覆貼圖
          reply = StickerMessage(
              package_id='446',
              sticker_id='1989'
            )
        elif action == 'image':
          #回覆圖片
          reply = ImageMessage(
              original_content_url='https://www.meowbox.com/cdn/shop/articles/Screen_Shot_2024-03-15_at_10.53.41_AM.png?v=1710525250',
              preview_image_url='https://www.meowbox.com/cdn/shop/articles/Screen_Shot_2024-03-15_at_10.53.41_AM.png?v=1710525250'
            )
        elif action == 'video':
          #回覆影片
          reply = VideoMessage(
              original_content_url='https://cdn.pixabay.com/video/2025/01/03/250395_large.mp4',
              preview_image_url='https://cdn.pixabay.com/photo/2015/04/23/22/00/tree-736885_1280.jpg'
            )
        elif action == 'location':
          reply = LocationMessage(
              title="中山大學",
              address="804高雄市鼓山區蓮海路70號",
              latitude=22.628655579154614,
              longitude=120.26482091633952
)
        else:
          response = ask_gemini(action)
          reply = TextMessage(text=response)

        line_bot_api.reply_message(
        ReplyMessageRequest(
          reply_token=event.reply_token,
          messages=[
            reply
            ]
          )
        )




ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://8dc8-35-221-223-189.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/May/2025 06:27:45] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 06:27:57] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 06:28:20] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 06:29:02] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 06:29:09] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 06:29:22] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 06:29:45] "POST /callback HTTP/1.1" 200 -
